## Imports

In [42]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Load the Data

In [43]:
df_games = pd.read_csv('../../data/steam_games.csv')
# df_reviews = pd.read_csv('../../data/adjusted/sample_sentiment.csv')
df_reviews = pd.read_csv('../../data/adjusted/steam_reviews_sentiment.csv')

## Additional data preprocessing

In [45]:
# Remove duplicate games by unique id
df_games = df_games.drop_duplicates(subset='id')

# View missing values for games
print("Games Missing Values:", end='\n\n')
print(df_games.isnull().sum())
# Remove games if they are missing:
# 1. price
# 2. reviews_url
# 3. id
df_games.dropna(subset=['price', 'reviews_url', 'id'], inplace=True)

# Users may have reviewed the same game multiple times so we will use the latest review:
df_reviews = df_reviews.sort_values(by=['username', 'product_id', 'date'], ascending=[True, True, False])
df_reviews = df_reviews.drop_duplicates(subset=['username', 'product_id'])

# View duplicate reviews based on username and product_id
# print(df_reviews[df_reviews.duplicated(subset=['username', 'product_id'], keep=False)]) - None Remain

# View missing values for these columns
print("Reviews Missing Values:", end='\n\n')
print(df_reviews[['username', 'product_id', 'rating_sentiment']].isnull().sum())

# We found missing usernames, so we will remove these reviews
df_reviews.dropna(subset=['username'], inplace=True)

# Remove whitespace from strings in 'username' and 'product_id'
df_reviews['username'] = df_reviews['username'].str.strip()
df_reviews['product_id'] = df_reviews['product_id'].astype(str).str.strip()

duplicates = df_reviews.duplicated(subset=['username', 'product_id']).sum()
print(f"Number of duplicate (username, product_id) pairs: {duplicates}")

Games Missing Values:

app_name              0
developer          3156
discount_price    30596
early_access          0
genres             3135
id                    0
metascore         28173
price                 0
publisher          7792
release_date       1936
reviews_url           0
sentiment          6584
specs               655
tags                130
title              1932
url                   0
dtype: int64
Reviews Missing Values:

username            0
product_id          0
rating_sentiment    0
dtype: int64
Number of duplicate (username, product_id) pairs: 0


## Create User Matrix

In [46]:
# We have alot of data. We need to restrict this to get more meaningful results.

# Total reviews
print(f"Total reviews: {len(df_reviews)}")
# Total users
active_users = df_reviews['username'].value_counts()
print(f"Total users from reviews: {len(active_users)}")

# Total games
active_games = df_reviews['product_id'].value_counts()
print(f"Total games from reviews: {len(active_games)}")

# Total games in the games dataset
print(f"Total games in the games dataset: {len(df_games)}")

min_reviews = 10
min_games = 100

# To get more meaningful results, let us ensure that the user has at least the minimum number of reviews
# and the game has at least the minimum number of reviews

df_reviews = df_reviews[ df_reviews['username'].isin(active_users[active_users > min_reviews].index) &
                         df_reviews['product_id'].isin(active_games[active_games > min_games].index)]

print(f"Total reviews after filtering: {len(df_reviews)}")

# Ensure product_id in df_reviews is treated as integers
reviewed_game_ids = df_reviews['product_id'].astype(int).unique()

# Filter df_games to keep only games that are reviewed
df_games_with_reviews = df_games[df_games['id'].isin(reviewed_game_ids)]

# Now df_games_with_reviews contains only games with at least one review
print(f"Number of games with reviews: {len(df_games_with_reviews)}")


Total reviews: 6843996
Total users from reviews: 2553983
Total games from reviews: 15471
Total games in the games dataset: 30755
Total reviews after filtering: 2064240
Number of games with reviews: 4786


In [51]:

df_user_matrix = df_reviews.pivot(index='username', columns='product_id', values='rating_sentiment')

# Games that have not been reviewed by a user must be filled with 0
df_user_matrix.fillna(0, inplace=True)

In [53]:
# View the user matrix
df_user_matrix.head()

product_id,10,10090,10100,10110,10130,10150,10180,10220,102200,102500,...,9900,99100,99200,9930,99300,9940,99400,99700,99900,99910
username,,,,,,,,,,,,,,,,,,,,,
! Taz the Husky,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!! Vooper !!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!!!~(. _ .✿ )~!!!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
